# Setup and Metrics

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 6.0)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.grid'] = True

## Floating-Point Types

In 1985, the Institute of Electrical and Electronics Engineers (IEEE) introduced the **IEEE 754** standard, which defines how computers represent and perform arithmetic on real numbers using a finite number of binary bits.

Most modern programming languages and hardware follow the IEEE 754 standard. For the purpose of this repository, Python and Jupyter notebooks are used, and all floating-point computations are performed using NumPy's IEEE-754 compliant data types: `np.float32` (single precision) and `np.float64` (double precision). These correspond directly to the standard 32-bit and 64-bit IEEE 754 floating-point formats and are used throughout the repository to study precision, rounding behavior, and numerical error.

### Normalized Floating-Point Representation
A (normalized) IEEE 754 floating-point number is represented as:  

$(-1)^{sign} \times 1.fraction \times 2^{exponent}$  

This mirrors scientific notation, but in base 2 instead of base 10.  

#### Components of a Floating-Point Number
**Sign Bit**  
A single bit that determines the sign of the number:  
`0` = Positive  
`1` = Negative  

**Mantissa (Fraction/Significand)**  
Stores the significant digits of the number. For normalized numbers, the leading 1 is implicit.  

**Exponent**  
Stores the scale of the number using a bias so that both positive and negative exponents can be represented.  

### Python (NumPy) Normalized Floating-Point Representation
NumPy's `np.float32` and `np.float64` types follow the IEEE 754 standard for normalized floating-point numbers, differing only in the number of bits allocated to the exponent and mantissa.  

`np.float32` (Single Precision)  
A normalized `np.float32` value is stored using 32 bits:  
Sign: 1 bit  
Exponent: 8 bits (bias = 127)  
Mantissa: 23 bits  
Numerical Representation: $(-1)^{sign} \times 1.fraction \times 2^{exponent-127}$  

`np.float64` (Double Precision)  
A normalized `np.float64` value is stored using 64 bits:  
Sign: 1 bit  
Exponent: 11 bits (bias = 1023)  
Mantissa: 52 bits  
Numerical Representation: $(-1)^{sign} \times 1.fraction \times 2^{exponent-1023}$  

In [5]:
DTYPES = [np.float32, np.float64]

def as_dtype(x, dtype):
    return x.astype(x, dtype=dtype)

### References
Wikipedia contributors. (2025, November 29). IEEE 754. In Wikipedia, The Free Encyclopedia. Retrieved 01:07, December 29, 2025, from https://en.wikipedia.org/w/index.php?title=IEEE_754&oldid=1324852683